In [1]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from frequency_based import TFIDFVectorizer #ETC Version
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import f1_score
from sklearn.pipeline import Pipeline
import numpy as np
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS as sk_stop_words
from nltk.corpus import stopwords as nltk_stop_words
nltk_stop_words = nltk_stop_words.words('english')
print(len(nltk_stop_words), len(sk_stop_words))

179 318


In [2]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import train_test_split

In [3]:
dataset = fetch_20newsgroups()

In [4]:
X_train, X_test, y_train, y_test = train_test_split(dataset.data, dataset.target, test_size=0.1, random_state=42)

In [5]:
parameters = {
    'tfidf__stopwords': (None, nltk_stop_words, sk_stop_words),
    'tfidf__min_df': (1,2,4),
    'tfidf__ngram_range': ((1,1), (1,2), (2,2)),
    'clf__C': 2.0 ** np.arange(-5, 8, 2)
}
init_params= {'kernel': 'linear', 'C': 1, 'verbose': False, 'probability': False,
     'degree': 3, 'shrinking': True, 
     'decision_function_shape': 'ovr', 'random_state': None, 
     'tol': 0.001, 'cache_size': 25000, 'coef0': 0.0, 'gamma': 'auto', 
     'class_weight': None,'random_state': 42}

In [6]:
pipeline = Pipeline([
    ('tfidf', TFIDFVectorizer()),
    ('clf', SVC(**init_params)),
])
pipeline2 = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', SVC(**init_params)),
])

In [7]:
grid_search = GridSearchCV(pipeline, parameters, cv=3, n_jobs=64, verbose=10)

In [ ]:
grid_search.fit(X_train, y_train)

Fitting 3 folds for each of 189 candidates, totalling 567 fits


In [ ]:
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

In [ ]:
est = grid_search.best_estimator_
y_pred = est.predict(X_test)

In [ ]:
f1_score(y_test, y_pred, average='micro')

In [ ]:
f1_score(y_test, y_pred, average='macro')